# Flight Delay Prediction

## Data Loading and Preprocessing

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
# Load the cleaned flight data
flights_df = pd.read_parquet("cleaned_flights.parquet")

In [ ]:
# Extract the hour from the 'DATE' column and create a new column 'DEPARTURE_HOUR'
flights_df['DEPARTURE_HOUR'] = flights_df['DATE'].dt.hour

In [ ]:
# Replace missing values in 'DAILY_SNOWFALL' with 0
flights_df['DAILY_SNOWFALL'] = flights_df['DAILY_SNOWFALL'].fillna(0)

In [ ]:
# Select only delayed flights from flights_df
delayed_flights = flights_df[flights_df['ARRIVAL_DELAY'] > 0].copy()

## scikit-learn Models

In [ ]:
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.preprocessing import PolynomialFeatures
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor

In [ ]:
# Standardize numeric columns for sklearn models
for col in ['MONTH', 'DEPARTURE_HOUR', 'DAY_OF_WEEK', 'DISTANCE']:
    col_mean = delayed_flights[col].mean()
    col_std  = delayed_flights[col].std()
    delayed_flights[col] = (delayed_flights[col] - col_mean) / col_std

In [ ]:
# Extract features for sklearn models
numeric_feats = delayed_flights[['MONTH', 'DEPARTURE_HOUR', 'DAY_OF_WEEK', 'DISTANCE', 'DAILY_SNOWFALL']].values
categorical_feats = pd.get_dummies(delayed_flights[['AIRLINE', 'origin_airport/AIRPORT', 'destination_airport/AIRPORT']]).values
X = np.hstack([numeric_feats, categorical_feats])
y = delayed_flights['ARRIVAL_DELAY'].values

In [ ]:
split = int(0.8 * X.shape[0])
X_train, X_test = X[:split], X[split:]
y_train, y_test = y[:split], y[split:]

In [ ]:
# Gradient Boosting without cross-validation
gb_model = GradientBoostingRegressor(n_estimators=100, learning_rate=0.1, random_state=42, verbose=3) # Using common parameters
gb_model.fit(X_train, y_train)

gb_predictions = gb_model.predict(X_test)
gb_mse = mean_squared_error(y_test, gb_predictions)
print("GB test MSE:", gb_mse)

In [ ]:
# Random Forest without cross-validation
rf_model = RandomForestRegressor(n_estimators=100, max_depth=10, random_state=42, verbose=3) # Using common parameters
rf_model.fit(X_train, y_train)

rf_predictions = rf_model.predict(X_test)
rf_mse = mean_squared_error(y_test, rf_predictions)
print("RF test MSE:", rf_mse)

In [ ]:
# Print the MSE, R2, and RMSE for both models
gb_r2 = r2_score(y_test, gb_predictions)
gb_rmse = np.sqrt(gb_mse)
rf_r2 = r2_score(y_test, rf_predictions)
rf_rmse = np.sqrt(rf_mse)
print(f"GB test R2: {gb_r2}, RMSE: {gb_rmse}")
print(f"RF test R2: {rf_r2}, RMSE: {rf_rmse}")